In [11]:
import catboost
import pandas as pd
import numpy as np
from typing import List
import pandas as pd
from scipy import stats
import datetime
import rpy2
from functools import reduce
from catboost import CatBoostRegressor
import time
import seaborn as sns
import ast
import numpy as np
from dateutil import parser
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import catboost
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

In [34]:
# Path fo import files
#input data
import_data = "import/bimbo_2000.csv"
#generated data
transformed_analyzed_data = 'export/bimbo/bimbo_initial_time_series_2000.csv'
monetary_time_series = "export/bimbo/bimbo_monetary_2000.csv"
frequency_time_series = "export/bimbo/bimbo_frequency_2000.csv"
recency_time_series = "export/bimbo/bimbo_recency_2000.csv"
cluster_centroids_time_series = "export/bimbo/bimbo_cluster_centroids_2000.csv"
#prediction
prediction_path = "export/bimbo/bimbo_prediction.csv"

## 0. Preprocessing bimbo

In [13]:
df = pd.read_csv(import_data)
df = df.drop(columns = ['Unnamed: 0'])
df = df.groupby(by=['uid','week']).sum().reset_index()
df = df.pivot(columns = 'week',index = 'uid',values = 'target')
df.to_csv(transformed_analyzed_data)
df.sample(5)                                 

week,1,2,3,4,5,6,7
uid,,,,,,,
2055248,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1279938,265.0,200.0,212.0,191.0,188.0,257.0,224.0
4110479,74.0,105.0,140.0,239.0,138.0,92.0,132.0
4374300,13.0,59.0,25.0,30.0,30.0,19.0,18.0
26385,14.0,0.0,2.0,16.0,1.0,4.0,6.0


## 1  RFM-analysis

In [14]:
df_recency =  pd.DataFrame(columns = df.columns)
df_monetary =  pd.DataFrame(columns = df.columns)
df_frequency = pd.DataFrame(columns = df.columns)

In [15]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry)):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
    #MONETARY  Moving average monetary value
    current_servers = 0
    total_days_of_purchases = 0 
    first_purchase = -1#flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_days_of_purchases/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

## 2. Clustering

In [16]:
ksh_monetary = KShape(n_clusters=4)
df_monetary['label'] = ksh_monetary.fit(df_monetary.values)
ksh_frequency = KShape(n_clusters=4)
df_frequency['label'] = ksh_frequency.fit(df_frequency.values)
ksh_recency = KShape(n_clusters=4)
df_recency['label'] = ksh_recency.fit(df_recency.values)

0.280 --> 0.200 --> 0.149 --> 0.125 --> 0.137 --> 
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster
Resumed because of empty cluster


In [17]:
df_monetary['label'] =ksh_monetary.labels_
df_frequency['label'] =ksh_frequency.labels_
df_recency['label'] =ksh_recency.labels_

In [18]:
df_centroids = pd.DataFrame(columns = df_recency.drop(columns = 'label').columns)
for i in range(ksh_recency.cluster_centers_.shape[0]):
    df_centroids.loc['Centroid_for_recency_'+str(i)]= ksh_recency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_frequency_'+str(i)]= ksh_frequency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_monetary_'+str(i)]= ksh_monetary.cluster_centers_[i].ravel()

In [19]:
df_monetary.to_csv(monetary_time_series)
df_frequency.to_csv(frequency_time_series)
df_recency.to_csv(recency_time_series)
df_centroids.to_csv(cluster_centroids_time_series)

## 3. Preprocessing for prediction

In [20]:
df = pd.read_csv(transformed_analyzed_data)
df.head()

,uid,1,2,3,4,5,6,7
0,2018,86.0,153.0,100.0,89.0,99.0,96.0,93.0
1,3945,48.0,28.0,40.0,22.0,32.0,27.0,43.0
2,5457,22.0,16.0,46.0,35.0,10.0,0.0,32.0
3,5861,0.0,0.0,55.0,0.0,0.0,0.0,0.0
4,6127,77.0,102.0,134.0,76.0,80.0,55.0,89.0


In [21]:
df = pd.melt(df,id_vars = ['uid'],var_name ='timestamp' ,value_name='value')
df.head()

,uid,timestamp,value
0,2018,1,86.0
1,3945,1,48.0
2,5457,1,22.0
3,5861,1,0.0
4,6127,1,77.0


In [22]:
df_freq = pd.read_csv(frequency_time_series).loc[:,['Unnamed: 0','label']]
df_monet = pd.read_csv(monetary_time_series).loc[:,['Unnamed: 0','label']]
df_recency = pd.read_csv(recency_time_series).loc[:,['Unnamed: 0','label']]

In [23]:
df_monet.head()

,Unnamed: 0,label
0,2018,1
1,3945,0
2,5457,0
3,5861,3
4,6127,0


In [24]:
df_combined = df.merge(df_recency,left_on = 'uid',right_on='Unnamed: 0') \
    .merge(df_freq,left_on = 'uid',right_on='Unnamed: 0') \
    .merge(df_monet,left_on = 'uid',right_on='Unnamed: 0') \
    .drop(columns =['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])
df_combined.columns = ['uid','timestamp','value','recency','frequency','monetary']
df_combined.sample(5)

,uid,timestamp,value,recency,frequency,monetary
10933,4370252,7,5.0,0,0,3
2127,350320,7,35.0,0,0,1
2040,328105,4,0.0,0,0,1
3186,570969,2,233.0,0,0,1
7181,1922080,7,120.0,0,0,1


In [25]:
#For bimbo: we will regard week as month for simplicity
df_combined['timestamp'] = df_combined['timestamp'].apply(lambda x:datetime.datetime.strptime(x, "%m"))
df_combined.sample(2)

,uid,timestamp,value,recency,frequency,monetary
8920,2380344,1900-03-01,126.0,0,0,3
555,87876,1900-03-01,0.0,0,0,0


In [26]:
df_combined = df_combined.fillna(0)

In [27]:
# number of months
test_period = 0
current_date = df_combined.timestamp.max()
df_test = df_combined[df_combined.timestamp >= current_date - pd.DateOffset(months = test_period )]
df_train = df_combined[df_combined.timestamp < current_date - pd.DateOffset(months = test_period )]

In [28]:
df_test.shape[0]/df_train.shape[0]

0.16666666666666666

In [29]:
df_train.loc[df_train['uid']==1423528]

,uid,timestamp,value,recency,frequency,monetary
5635,1423528,1900-01-01,6.0,0,0,0
5636,1423528,1900-02-01,6.0,0,0,0
5637,1423528,1900-03-01,8.0,0,0,0
5638,1423528,1900-04-01,7.0,0,0,0
5639,1423528,1900-05-01,2.0,0,0,0
5640,1423528,1900-06-01,4.0,0,0,0


In [30]:
df_test.loc[df_test['uid']==1423528]

,uid,timestamp,value,recency,frequency,monetary
5641,1423528,1900-07-01,7.0,0,0,0


In [31]:
y_train = df_train['value']
y_test = df_test['value']

X_train = df_train.drop('value', axis = 1)
X_test = df_test.drop('value', axis = 1)

## 4. Prediction

In [32]:
start_time = time.time()

cv = KFold(n_splits = 3, shuffle = True, random_state = 42)  

model = CatBoostRegressor(verbose = False, 
                           random_state=42)

param_grid = {
              'n_estimators': [200, 300],
              'max_depth': [3, 4], 
              'learning_rate': [0.03, 0.05, 0.07] 
             }

gs_cb = GridSearchCV(model, 
                  param_grid,
                  cv = cv,
                  scoring = 'neg_mean_squared_error')

gs_cb.fit(X_train, y_train)

#train_RMSE = np.sqrt(-gs_cb.cv_results_['mean_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
#train_std = np.sqrt(gs_cb.cv_results_['std_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

cv_RMSE = np.sqrt(-gs_cb.cv_results_['mean_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
cv_std = np.sqrt(gs_cb.cv_results_['std_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

test_RMSE = np.sqrt(mean_squared_error(gs_cb.predict(X_test), y_test))

#print('train RMSE: {} +/- {}'.format(np.round(train_RMSE, 2), np.round(train_std, 2)))
print('CV RMSE: {} +/- {}'.format(np.round(cv_RMSE, 2), np.round(cv_std, 2)))
print('test RMSE: {}'.format(np.round(test_RMSE, 2)))
print(gs_cb.best_params_)
print('time :{} min'.format(np.round((time.time() - start_time)/60, 2))) 

#filename = 'CatBoost_bimbo.sav'
#pickle.dump(gs_cb, open(filename, 'wb'))

CV RMSE: 305.15 +/- 171.08
test RMSE: 278.55
{'learning_rate': 0.07, 'max_depth': 4, 'n_estimators': 300}
time :7.15 min


In [35]:
result = X_test
result['predicted_value']=gs_cb.predict(X_test)
result['true_value']=y_test


result.to_csv(prediction_path)